# NoteBook pour créer différents Datasets

### IMPORTS

In [238]:
using CSV, DataFrames, Statistics, Dates, Gadfly, LinearAlgebra, Distributions, Random, ScikitLearn, GLM

# Fonctions Globales

In [239]:
"""
    splitdataframe(df::DataFrame, p::Real)

Partitionne en un ensemble d'entraînement et un ensemble de validation un DataFrame.

### Arguments
- `df::DataFrame` : Un DataFrame
- `p::Real` : La proportion (entre 0 et 1) de données dans l'ensemble d'entraînement.

### Détails

La fonction renvoie deux DataFrames, un pour l'ensemble d'entraînement et l'autre pour l'ensemble de validation.

### Exemple

\```
 julia> splitdataframe(df, p.7)
\```

"""
function splitdataframe(df::DataFrame, p::Real)
   @assert 0 <= p <= 1 
    
    n = size(df,1)
    
    ind = shuffle(1:n)
    
    threshold = Int64(round(n*p))
    
    indTrain = sort(ind[1:threshold])
    
    indTest = setdiff(1:n,indTrain)
    
    dfTrain = df[indTrain,:]
    dfTest = df[indTest,:]
    
    return dfTrain, dfTest
    
end

splitdataframe

### Fonction pour compter le nombre de valeurs "missing" dans une ligne d'un dataframe

In [240]:
function countMissing(line)
    count=0
    ind = 0
    for i=1:length(line)
        if (ismissing(line[i]))
            count += 1
            ind = i
        end
    end
    return count, ind
end


countMissing (generic function with 1 method)

### Fonction permettant de calculer le moyenne des valeurs d'une ligne peu importe le nombre de "missing"

In [241]:
function meanLine(line)
    values = []
    for i=1:length(line)
        if (!ismissing(line[i]))
            append!(values,line[i])
        end
    end
    return mean(values)
end


meanLine (generic function with 1 method)

### Fonction d'encodage OneHot

In [242]:
function oneHot(df_grouped)
    oneHotMatrix = []
    dateVector = []
    oneHotDF = DataFrame()
    for df in df_grouped
        vector = zeros(5)
        for row = eachrow(df)
            i = findfirst(x -> x==row[:NO_OUVRAGE], OUVRAGES)
            vector[i] = row[:SURVERSE]
        end
        push!(dateVector, df[:DATE][1])
        push!(oneHotMatrix, vector)
    end
    oneHotDF.one_hot = oneHotMatrix
    oneHotDF.date = dateVector
    return oneHotDF
end

oneHot (generic function with 1 method)

### Fonction permettant de remplacer toutes les valeurs "missing" d'une ligne avec la valeur donnée en parametre

In [243]:
function replaceMissing(line,value)
    n = length(line)
    for i=1:n
        if (ismissing(line[i]))
            line[i] = value
        end
    end
end

replaceMissing (generic function with 1 method)

# Chargement des données et nettoyage préliminaire

## Chargement des surverses

In [244]:
data = CSV.read("./data/surverses.csv", missingstring="-99999")
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


## Nettoyage des données sur les surverses

#### Extraction des surverses pour les mois de mai à octobre inclusivement

In [245]:
data = filter(row -> month(row.DATE) > 4, data) 
data = filter(row -> month(row.DATE) < 11, data) 
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


#### Remplacement des valeurs *missing* dans la colonne :RAISON par "Inconnue"

In [246]:
raison = coalesce.(data[:,:RAISON],"Inconnue")
data[!,:RAISON] = raison
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String
1,0642-01D,2013-05-01,0,Inconnue
2,0642-01D,2013-05-02,0,Inconnue
3,0642-01D,2013-05-03,0,Inconnue
4,0642-01D,2013-05-04,0,Inconnue
5,0642-01D,2013-05-05,0,Inconnue


#### Exlusion des surverses coccasionnées par d'autres facteurs que les précipitations liquides

Ces facteurs correspondent à : 
- la fonte de neige (F), 
- les travaux planifiés et entretien (TPL)
- urgence (U)
- autre (AUT)

In [247]:
data = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data) 
select!(data, [:NO_OUVRAGE, :DATE, :SURVERSE])
first(data,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64⍰
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


#### Exclusion des lignes où :SURVERSE est manquante

In [248]:
surverse_df = dropmissing(data, disallowmissing=true)
rename!(surverse_df, :DATE=>:date)
first(surverse_df,5)

,NO_OUVRAGE,date,SURVERSE
,String,Date,Int64
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


## Chargement des précipitations

In [249]:
data = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data, Symbol("St-Hubert")=>:StHubert)
first(data,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-01-01,0,0,0,0,0,missing
2,2013-01-01,1,0,0,0,0,missing
3,2013-01-01,2,0,0,0,0,missing
4,2013-01-01,3,0,0,0,0,missing
5,2013-01-01,4,0,0,0,0,missing


## Nettoyage des données sur les précipitations

#### Extraction des précipitations des mois de mai à octobre inclusivement

In [250]:
data = filter(row -> month(row.date) > 4, data) 
data = filter(row -> month(row.date) < 11, data) 
first(data,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,0,missing
2,2013-05-01,1,0,0,0,0,missing
3,2013-05-01,2,0,0,0,0,missing
4,2013-05-01,3,0,0,0,0,missing
5,2013-05-01,4,0,0,0,0,missing


 ### Remplissage des données manquantes
Nous allons tenter de remplir les données manquantes par des moyennes de précipitations lorsque les données sont inconnues pour 2 stations ou plus afin d'avoir plus de données pour créer nos modèles

# Techniques pour remplir les données manquantes

- utiliser regression ridge pour remplir les ligne ou il y a une valeur manquante 
- remplir les lignes avec plusieurs valeurs manquantes avec la moyenne des valeurs présentes

## Fonction Ridge pour trouver les valueurs manquantes

In [251]:
# Fonction pour faire une regression ridge
# Ressort le beta, m, et s
function ridge(datas::DataFrame, station::Symbol)
       
    Train, Test = splitdataframe(datas, .75);
    # Prétraitement des données
    # Les variables avec les tildes correspondent à l'échantillon de test

    X = convert(Matrix{Int64},Train[:,Not(station)])
    m = mean(X, dims=1)
    s = std(X, dims=1)
    m[2] = 0
    s[2] = 1
    X = (X .- m) ./ s

    X̃ = convert(Matrix{Int64},Test[:,Not(station)])
    X̃ = (X̃ .- m) ./ s

    y = convert(Vector{Int64}, Train[:,station])
    m = mean(y)
    s = std(y)
    y = (y .- m) ./s

    ỹ = convert(Vector{Int64}, Test[:,station])
    ỹ = (ỹ .- m) ./s;

    #On calcule ensuite le RMSE pour chacun des valeurs de lambda
    RMSEs = DataFrame(λ=Float64[], RMSE=Float64[])

    for λ in 0:1:10000
   
        β̂ = (X'X + λ*I)\X'y
    
        ŷ = X̃*β̂
    
        ẽ = ỹ - ŷ
    
        RMSE = sqrt(dot(ẽ,ẽ)/length(ẽ))
    
        push!(RMSEs, [λ, RMSE])
    
    end
    
    # On trouve ensuite la valeure de lambda qui minimise le RMSE
    _, ind = findmin(RMSEs[:,:RMSE])

    λ̂ = RMSEs[ind,:λ]
    
    β̂ = (X'X + λ̂*I)\X'y
    
    #TODO validate model and print value of validator R² ajuste
    
    #On peut alors calculer les y avec les betas trouver et l'echantillon de test
    ŷ = X̃ * β̂
    ŷ = round.((ŷ .* s) .+ m)
    
    # Calcul du R² ajusté

    p = 4          # nombre de variables explicatives
    n = length(ỹ)  # taille de l'échantillon

    ỹ = (ỹ .* s) .+ m
    ȳ = mean(ỹ)
    e = ỹ - ŷ

    SST = sum( (ỹ[i] - ȳ)^2 for i=1:n )  # variabilité totale
    SSE = sum( e.^2 )                    # variabilité résiduelle

    R2aj =  1 - SSE/SST * (n-1)/(n-p)
    
    println("Le R² ajuste du modele trouve pour la station de $(station) est $(R2aj)")
    
    return β̂
end

ridge (generic function with 1 method)

In [252]:
# utilisations des données connues pour bâtire les modèles de prédiction Ridge
full_data = dropmissing(data, disallowmissing=true)
full_data = full_data[:,Not(:date)][:, Not(:heure)]
size(full_data)

(20001, 5)

In [ ]:
# Liste des betas par station manquante (prédite)
betas = DataFrame(station = Symbol[], β = Array{Float64}[])
for name in names(full_data)
    β̂ = ridge(full_data, name)
    push!(betas, [name, β̂])
end

## Remplissage des données en effectuant la moyenne des données présentes si plus de 1 est manquante, sinon utilisation du modèle Ridge approprié à la station manquante

In [ ]:
precipitation_df = data[:,Not(:date)][:,Not(:heure)]
for row in eachrow(precipitation_df)
    nbMissing, ind = countMissing(row)
    if(nbMissing == 1)
        row[ind] = round((convert(Vector{Float64},row[Not(ind)])'*betas[:, :β][ind]))
    end
    # remplacer les lignes qui ont de 2 a 4 missing
    if(nbMissing<5 && nbMissing>1)
        replaceMissing(row,round(meanLine(row)))
    end
end
precipitation_df.heure = data[:,:heure]
precipitation_df.date = data[:,:date]
precipitation_df = dropmissing(precipitation_df) # drop all missing
CSV.write("data/new_datasets/precipitation_filed_mean_per_hour.csv",precipitation_df)
first(precipitation_df,10)

## Nouvelle variable explicative : somme des précipitations par jour pour les stations de précipitations
- Nous pensons que cela pourrait bien expliquer des surverses

In [ ]:
precipitation_daily_sum = by(precipitation_df, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
   Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)
last(precipitation_daily_sum ,10)

### Extraire l'année 2019 pour les données à prédire

In [ ]:
precipitation_daily_sum_train = filter(row -> Year(row[:date]) != Year(2019), precipitation_daily_sum)
precipitation_daily_sum_pred  = filter(row -> Year(row[:date]) == Year(2019), precipitation_daily_sum)

#### Envoyer vers fichier CSV

In [ ]:
filter!(row -> row.date in surverse_df[!, :date], precipitation_daily_sum_train)
filter!(row -> row.date in precipitation_daily_sum_train[!, :date], surverse_df)

In [ ]:
CSV.write("data/new_datasets/precipitation_daily_sum/x_train.csv", precipitation_daily_sum_train)
CSV.write("data/new_datasets/precipitation_daily_sum/x_pred.csv", precipitation_daily_sum_pred)

## Nouvelle variable explicative : Maximum des précipitations par jour pour les stations de précipitations
- Nous pensons que cela pourrait bien expliquer des surverses

#### Extraction du taux horaire journalier maximum des précipitations pour chacune des stations météorologiques

In [ ]:
precipitation_daily_max = by(precipitation_df, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)
first(precipitation_daily_max,10)

### Extraire l'année 2019 pour les données à prédire


In [ ]:
precipitation_daily_max_train = filter(row -> Year(row[:date]) != Year(2019), precipitation_daily_max)
precipitation_daily_max_pred  = filter(row -> Year(row[:date]) == Year(2019), precipitation_daily_max)

#### Envoyer vers fichier CSV
Valider que les dates correspondent pour x et y

In [ ]:
filter!(row -> row.date in surverse_df[!, :date], precipitation_daily_max_train)
filter!(row -> row.date in precipitation_daily_max_train[!, :date], surverse_df)

In [ ]:
CSV.write("./data/new_datasets/precipitation_daily_max/x_train.csv", precipitation_daily_max_train)
CSV.write("./data/new_datasets/precipitation_daily_max/x_pred.csv", precipitation_daily_max_pred)

In [ ]:
CSV.write("./data/new_datasets/surverse_list.csv", surverse_df)

## Nouvelle variable explicative : somme plus importante de précipitations par plages horaires sur une journée 
- Nous avons extrait cette varibale explicative avec l'idéé en tête que quelsques heures consécutives avec beaucoup de précipitations pourraient causer des surverses plus que des précipitations modérées réparties sur une journée complète

In [ ]:
filledPrec  = CSV.read("data/new_datasets/precipitation_filed_mean_per_hour.csv", missingstring="-99999")
first(filledPrec,5)

#### Fonctions pour séparer les journées en plages horaires de taille "window" et effectuer la somme des précipiations sur chaque plage

In [ ]:
function dayPrecipitationSplit(window,precipitations)
    n = size(precipitations,1)
    newDf = DataFrame(McTavish = Int64[], Bellevue = Int64[], Assomption = Int64[], Trudeau  = Int64[],
                        StHubert = Int64[], heureDebut = Int64[], heureFin = Int64[], date = Date[])

    hourgroups = 24/window
    for day in groupby(filledPrec, :date)
        start = 1
        finish = window
        date = day[1,:date]
        if(size(day, 1)==24)
            for i=1:hourgroups
                mcTavish, Bellevue, Assomption, Trudeau, StHubert, heureDebut, heureFin = 0,0,0,0,0,0,0
                for j=start:finish
                    mcTavish += day[j,:McTavish]
                    Assomption += day[j,:Assomption]
                    Bellevue += day[j,:Bellevue]
                    Trudeau += day[j,:Trudeau]
                    StHubert += day[j,:StHubert]
                    if j == start
                        heureDebut = day[j,:heure]
                    elseif j == finish
                        heureFin = day[j,:heure]
                    end
                end
                start += window
                finish += window
                push!(newDf,[mcTavish,Bellevue,Assomption,Trudeau,StHubert,heureDebut,heureFin,date])
            end
        end
    end
    return newDf
end

#### Générations des sommes de plages horaires pour des divisions de journées de 2,3,4,6,8 et 12 heures

In [ ]:
hourSplit2 = dayPrecipitationSplit(2,filledPrec)
hourSplit3 = dayPrecipitationSplit(3,filledPrec)
hourSplit4 = dayPrecipitationSplit(4,filledPrec)
hourSplit6 = dayPrecipitationSplit(6,filledPrec)
hourSplit8 = dayPrecipitationSplit(8,filledPrec)
hourSplit12 = dayPrecipitationSplit(12,filledPrec)
first(hourSplit12,5)

#### Fonction pour extraire la plage horaire avec la somme des précipitations la plus grande pour chaque jour

In [ ]:
function maxPrecByDay(Prec)
    n = size(Prec,1)
    newDf = DataFrame(McTavish = Int64[], Bellevue = Int64[], Assomption = Int64[], Trudeau  = Int64[],
                        StHubert = Int64[], date = Date[])

    for day in groupby(Prec, :date)
        mcTavish = maximum(day[:,:McTavish])
        Assomption = maximum(day[:,:Assomption])
        Bellevue = maximum(day[:,:Bellevue])
        Trudeau = maximum(day[:,:Trudeau])
        StHubert = maximum(day[:,:StHubert])
        date = day[1,:date]
        push!(newDf,[mcTavish,Bellevue,Assomption,Trudeau,StHubert,date])
    end
    return newDf
end

#### Datasets contenant la nouvelle variables explicative pour les 5 stations pour les journées divisée par sections de 2,3,4,6,8 et 12 heures 

In [ ]:
maxSum2hours = maxPrecByDay(hourSplit2)
maxSum3hours = maxPrecByDay(hourSplit3)
maxSum4hours = maxPrecByDay(hourSplit4)
maxSum6hours = maxPrecByDay(hourSplit6)
maxSum8hours = maxPrecByDay(hourSplit8)
maxSum12hours = maxPrecByDay(hourSplit12)
first(maxSum12hours,5)

#### Écriture des dataframes ver des fichiers CSV

In [ ]:
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy2hours.csv",maxSum2hours)
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy3hours.csv",maxSum3hours)
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy4hours.csv",maxSum4hours)
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy6hours.csv",maxSum6hours)
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy8hours.csv",maxSum8hours)
CSV.write("./data/new_datasets/max_precipitation_day_split/maxPrecBy12hours.csv",maxSum12hours)

## Nouvelle variable explicative : Somme de la journee + les deux dernieres heures du jour précédent
- Nous avons décidé d'ajouter les deux heures de la journée précedente, car nous croyons que les dernières heures de cette journée peuvent effet impoartant sur la journée qui suit et pas sur celle dont elles font parite

In [ ]:
filledPrec  = CSV.read("data/new_datasets/precipitation_filed_mean_per_hour.csv", missingstring="-99999")
first(filledPrec,5)

#### Somme et ajout des 2 dernières heures de la journée précédente pour les précipitations

In [ ]:
for i=1:size(precipitation_daily_sum, 1)
    ind = findfirst(filledPrec[:,:date] .== precipitation_daily_sum[i,:date])
    for h=1:2
        for key in names(precipitation_daily_sum[:, Not(:date)])
            if ind-h > 0
                precipitation_daily_sum[i, key] += filledPrec[ind-h, key]
            end
        end
    end
end
first(precipitation_daily_sum, 5)

#### Exportation vers CSV

In [ ]:
CSV.write("./data/new_datasets/sum_day_last_2.csv",precipitation_daily_sum)

##### Dataset avec toutes les surverses des autres ouvrages ayant assez de données par date
- Nous n'avons finalement pas utilisé ces données, car il y avait une quatité très importante de données manquantes et nous n'avions pas accès aux données pour 2019 afin d'effectuer les préditions

In [ ]:
surversesVoisines = DataFrame()
surversesVoisines[:,:date] = maxSum2hours[:,:date]
ouvrages = groupby(surverse_df, :NO_OUVRAGE)
for subdf in ouvrages
            length = (size(subdf, 1))
            if (length > 1000)
                subdf = rename(subdf,:SURVERSE => subdf[1,:NO_OUVRAGE])
                dateAndSurverse = subdf[:,Not(:NO_OUVRAGE)]
                surversesVoisines = join(surversesVoisines, dateAndSurverse, on = :date, kind = :left)    
            end
       end
first(surversesVoisines,5)
CSV.write("./data/new_datasets/surversesVoisines",surversesVoisines)
